In [83]:
import numpy as np
import pandas as pd
import mgrs

In [62]:
zip_lat_lon = pd.read_csv('zip_lat_lon.csv')
zip_lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43191 entries, 0 to 43190
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip_5                       43191 non-null  int64  
 1   Zip                         43191 non-null  int64  
 2   City                        43191 non-null  object 
 3   State                       43191 non-null  object 
 4   Latitude                    43191 non-null  float64
 5   Longitude                   43191 non-null  float64
 6   Timezone                    43191 non-null  int64  
 7   Daylight savings time flag  43191 non-null  int64  
 8   geopoint                    43191 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 3.0+ MB


In [63]:
zip_lat_lon.head()

,zip_5,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,38732,38732,Cleveland,MS,33.749149,-90.71329,-6,1,33.749149
1,47872,47872,Rockville,IN,39.758142,-87.17540,-5,0,39.758142
2,50049,50049,Chariton,IA,41.028910,-93.29857,-6,1,41.028910
3,48463,48463,Otisville,MI,43.167457,-83.52542,-5,1,43.167457
4,51063,51063,Whiting,IA,42.137272,-96.16648,-6,1,42.137272


In [64]:
zip_lat_lon = zip_lat_lon[['zip_5','City','State','Latitude','Longitude']]
zip_lat_lon = zip_lat_lon[zip_lat_lon['State']=='CA']
zip_lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2716 entries, 32 to 43172
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zip_5      2716 non-null   int64  
 1   City       2716 non-null   object 
 2   State      2716 non-null   object 
 3   Latitude   2716 non-null   float64
 4   Longitude  2716 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 127.3+ KB


In [65]:
pop_density = pd.read_csv('pop_density_by_zip.csv')
pop_density.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33144 entries, 0 to 33143
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_5                33144 non-null  int64  
 1   Zip/ZCTA             33144 non-null  int64  
 2   2010 Population      33144 non-null  int64  
 3   Land-Sq-Mi           33144 non-null  float64
 4   Density Per Sq Mile  33144 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 1.3 MB


In [66]:
pop_density_coords = zip_lat_lon.merge(pop_density,on='zip_5',how='inner')
pop_density_coords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1744 entries, 0 to 1743
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_5                1744 non-null   int64  
 1   City                 1744 non-null   object 
 2   State                1744 non-null   object 
 3   Latitude             1744 non-null   float64
 4   Longitude            1744 non-null   float64
 5   Zip/ZCTA             1744 non-null   int64  
 6   2010 Population      1744 non-null   int64  
 7   Land-Sq-Mi           1744 non-null   float64
 8   Density Per Sq Mile  1744 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 136.2+ KB


In [67]:
pop_density_coords.head()

,zip_5,City,State,Latitude,Longitude,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,92399,Yucaipa,CA,34.033564,-117.037760,92399,52606,53.035,991.911002
1,95957,Meridian,CA,39.070323,-121.830650,95957,761,69.859,10.893371
2,93225,Frazier Park,CA,34.815266,-119.004040,93225,5077,177.186,28.653505
3,90094,Los Angeles,CA,33.786594,-118.298662,90094,5464,0.611,8942.716858
4,92683,Westminster,CA,33.751418,-117.993920,92683,89747,9.991,8982.784506


In [68]:
# limit to only the MGRS locations in Southern CA
MGRS_100KM_USED = ['11SMT','11SNT','11SNS','11SMS','11SLT']

m = mgrs.MGRS() #mgrs API
to_mgrs_10km = []  #precision 1
to_mgrs_100km = []  #precision 0

for i, j in zip(pop_density_coords['Latitude'].tolist(), pop_density_coords['Longitude'].tolist()):
    to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
    to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) 

pop_density_coords['mgrs_10km'] = to_mgrs_10km
pop_density_coords['mgrs_100km'] = to_mgrs_100km

# limit to only the MGRS locations in Southern CA
pop_density_coords = pop_density_coords[pop_density_coords.mgrs_100km.isin(MGRS_100KM_USED)]
pop_density_coords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 1740
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_5                592 non-null    int64  
 1   City                 592 non-null    object 
 2   State                592 non-null    object 
 3   Latitude             592 non-null    float64
 4   Longitude            592 non-null    float64
 5   Zip/ZCTA             592 non-null    int64  
 6   2010 Population      592 non-null    int64  
 7   Land-Sq-Mi           592 non-null    float64
 8   Density Per Sq Mile  592 non-null    float64
 9   mgrs_10km            592 non-null    object 
 10  mgrs_100km           592 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 55.5+ KB


In [69]:
pop_density_coords.head(10)

,zip_5,City,State,Latitude,Longitude,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile,mgrs_10km,mgrs_100km
0,92399,Yucaipa,CA,34.033564,-117.037760,92399,52606,53.035,991.911002,11SMT96,11SMT
3,90094,Los Angeles,CA,33.786594,-118.298662,90094,5464,0.611,8942.716858,11SLT73,11SLT
4,92683,Westminster,CA,33.751418,-117.993920,92683,89747,9.991,8982.784506,11SMT03,11SMT
6,91730,Rancho Cucamonga,CA,34.104555,-117.587340,91730,66925,14.483,4620.934889,11SMT47,11SMT
8,92614,Irvine,CA,33.680408,-117.825920,92614,24748,4.420,5599.095023,11SMT22,11SMT
12,92655,Midway City,CA,33.745166,-117.985690,92655,8337,0.632,13191.455700,11SMT03,11SMT
14,90079,Los Angeles,CA,33.786594,-118.298662,90079,0,0.010,0.000000,11SLT73,11SLT
15,90670,Santa Fe Springs,CA,33.933551,-118.068210,90670,14866,8.462,1756.795084,11SMT05,11SMT
16,90742,Sunset Beach,CA,33.718167,-118.071790,90742,831,0.184,4516.304348,11SMT03,11SMT
17,91436,Encino,CA,34.155625,-118.489250,91436,14372,5.695,2523.617208,11SLT68,11SLT


In [70]:
pop_density_coords['mgrs_100km'].unique()

array(['11SMT', '11SLT', '11SMS', '11SNT', '11SNS'], dtype=object)

In [71]:
len(pop_density_coords['mgrs_10km'].unique())

201

In [72]:
mgrs_final = pd.read_csv('mgrs_final.csv')
mgrs_final = mgrs_final[['mgrs_10km','lat_10km','lon_10km']]
mgrs_final.head()

,mgrs_10km,lat_10km,lon_10km
0,11SMT03,33.705267,-118.079125
1,11SMT04,33.795444,-118.080255
2,11SMT05,33.885620,-118.081391
3,11SMT06,33.975794,-118.082531
4,11SMT07,34.065966,-118.083677


In [73]:
pop_density_mgrs = mgrs_final.merge(pop_density_coords,on='mgrs_10km',how='left')
pop_density_mgrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776 entries, 0 to 775
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mgrs_10km            776 non-null    object 
 1   lat_10km             776 non-null    float64
 2   lon_10km             776 non-null    float64
 3   zip_5                592 non-null    float64
 4   City                 592 non-null    object 
 5   State                592 non-null    object 
 6   Latitude             592 non-null    float64
 7   Longitude            592 non-null    float64
 8   Zip/ZCTA             592 non-null    float64
 9   2010 Population      592 non-null    float64
 10  Land-Sq-Mi           592 non-null    float64
 11  Density Per Sq Mile  592 non-null    float64
 12  mgrs_100km           592 non-null    object 
dtypes: float64(9), object(4)
memory usage: 84.9+ KB


In [74]:
pop_density_mgrs.head(10)

,mgrs_10km,lat_10km,lon_10km,zip_5,City,State,Latitude,Longitude,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile,mgrs_100km
0,11SMT03,33.705267,-118.079125,92683.0,Westminster,CA,33.751418,-117.99392,92683.0,89747.0,9.991,8982.784506,11SMT
1,11SMT03,33.705267,-118.079125,92655.0,Midway City,CA,33.745166,-117.98569,92655.0,8337.0,0.632,13191.455700,11SMT
2,11SMT03,33.705267,-118.079125,90742.0,Sunset Beach,CA,33.718167,-118.07179,90742.0,831.0,0.184,4516.304348,11SMT
3,11SMT03,33.705267,-118.079125,92841.0,Garden Grove,CA,33.786915,-117.98224,92841.0,32845.0,4.480,7331.473214,11SMT
4,11SMT03,33.705267,-118.079125,92649.0,Huntington Beach,CA,33.720017,-118.04614,92649.0,32463.0,7.638,4250.196386,11SMT
5,11SMT03,33.705267,-118.079125,90720.0,Los Alamitos,CA,33.794028,-118.07218,90720.0,21751.0,6.122,3552.923881,11SMT
6,11SMT03,33.705267,-118.079125,92845.0,Garden Grove,CA,33.782916,-118.02619,92845.0,16333.0,2.189,7461.397899,11SMT
7,11SMT03,33.705267,-118.079125,92844.0,Garden Grove,CA,33.765516,-117.97310,92844.0,24307.0,2.127,11427.832630,11SMT
8,11SMT03,33.705267,-118.079125,92647.0,Huntington Beach,CA,33.725167,-118.00509,92647.0,57245.0,7.748,7388.358286,11SMT
9,11SMT04,33.795444,-118.080255,90621.0,Buena Park,CA,33.875714,-117.99404,90621.0,35153.0,4.154,8462.445835,11SMT


In [75]:
pop_density_mgrs.tail(10)

,mgrs_10km,lat_10km,lon_10km,zip_5,City,State,Latitude,Longitude,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile,mgrs_100km
766,11SMS81,32.627384,-117.213205,92118.0,Coronado,CA,32.682727,-117.174410,92118.0,23575.0,7.756,3039.582259,11SMS
767,11SMS81,32.627384,-117.213205,92155.0,San Diego,CA,32.671602,-117.165665,92155.0,550.0,0.223,2466.367713,11SMS
768,11SMS81,32.627384,-117.213205,92113.0,San Diego,CA,32.697402,-117.120010,92113.0,56066.0,5.269,10640.728790,11SMS
769,11SMS66,33.077858,-117.428568,92008.0,Carlsbad,CA,33.158137,-117.326460,92008.0,27649.0,10.307,2682.545843,11SMS
770,11SMS71,32.627158,-117.319806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
771,11SMT11,33.525799,-117.969206,92662.0,Newport Beach,CA,33.606271,-117.892430,92662.0,2756.0,0.208,13250.000000,11SMT
772,11SMT11,33.525799,-117.969206,92661.0,Newport Beach,CA,33.601822,-117.899660,92661.0,3744.0,0.651,5751.152074,11SMT
773,11SMT11,33.525799,-117.969206,92625.0,Corona Del Mar,CA,33.600172,-117.868230,92625.0,12478.0,2.565,4864.717349,11SMT
774,11SMT20,33.436408,-117.860635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775,11SMS57,33.167642,-117.536254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
pop_missing = pop_density_mgrs[pop_density_mgrs['Longitude'].isna()]
pop_missing = pop_missing[['mgrs_10km','lat_10km','lon_10km']]
pop_missing.reset_index(drop=True,inplace=True)
pop_missing.head()

,mgrs_10km,lat_10km,lon_10km
0,11SMT09,34.246308,-118.085983
1,11SMT18,34.157049,-117.976359
2,11SMT19,34.247222,-117.977399
3,11SMT28,34.157863,-117.867886
4,11SMT29,34.248039,-117.868811


In [77]:
# pop_missing.to_csv('pop_missing.csv')

all the missing appear to be in the ocean or in the high mountains.  another large area where camp pendleton is.  it looks ok

In [78]:
pop_density_mgrs.groupby(['mgrs_10km'])['Density Per Sq Mile'].sum()

mgrs_10km
11SLT07       2.473607
11SLT08    2820.689420
11SLT09    4244.793942
11SLT16       0.000000
11SLT17       0.000000
              ...     
11SNT95       0.000000
11SNT96       0.000000
11SNT97       0.000000
11SNT98       0.000000
11SNT99       0.000000
Name: Density Per Sq Mile, Length: 385, dtype: float64

In [79]:
pop_density_mgrs[pop_density_mgrs['mgrs_10km']=='11SLT16']

,mgrs_10km,lat_10km,lon_10km,zip_5,City,State,Latitude,Longitude,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile,mgrs_100km
753,11SLT16,33.963371,-119.056389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
pop_density_mgrs = pd.DataFrame(pop_density_mgrs.groupby(['mgrs_10km'])['Density Per Sq Mile'].sum())
pop_density_mgrs.reset_index(inplace=True)
pop_density_mgrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mgrs_10km            385 non-null    object 
 1   Density Per Sq Mile  385 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.1+ KB


In [81]:
pop_density_mgrs

,mgrs_10km,Density Per Sq Mile
0,11SLT07,2.473607
1,11SLT08,2820.689420
2,11SLT09,4244.793942
3,11SLT16,0.000000
4,11SLT17,0.000000
...,...,...
380,11SNT95,0.000000
381,11SNT96,0.000000
382,11SNT97,0.000000
383,11SNT98,0.000000


In [84]:
np.sum(pop_density_mgrs['Density Per Sq Mile'])

3543709.962151825

In [85]:
np.sum(pop_density_coords['Density Per Sq Mile'])

3543709.9621518254

In [86]:
pop_density_mgrs.to_csv('pop_density_mgrs.csv')